

```
# This is formatted as code
```

Followed this tutorial: https://towardsdatascience.com/a-deep-learning-approach-in-predicting-the-next-word-s-7b0ee9341bfe
This is pretty basic next-word prediction from training on the NERs as sentences, it doesn't really use the embeddings.

In [1]:
import pickle
import numpy as np
import pandas
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd "/content/drive/My Drive/dis study abroad/Artificial Neural Networks"


Mounted at /content/drive
/content/drive/My Drive/dis study abroad/Artificial Neural Networks


In [3]:
%ls

emily_ingredients_model.ipynb     full_dataset.csv.zip     NLG_cleaning.ipynb
flavor_graph.ipynb                ingredients_model.ipynb  nodes_191120.csv
FlavorGraph_NodeEmbedding.pickle  ner_embeddings.pkl


In [4]:
# open and load data from pickle file
file = open('FlavorGraph_NodeEmbedding.pickle', 'rb')

# keys are strings for node_id, value is embedding
data = pickle.load(file)

file.close()

In [5]:
# load dataframe of node_ids and ingredients
df = pandas.read_csv('nodes_191120.csv')

In [6]:
# just get the embeddings for ingredients
ing_embeddings = {}

for i in range(len(df)):
    if df.loc[i, 'node_type'] == "ingredient":
        # map the name of the ingredient to the embedding
        ing_embeddings[df.loc[i, 'name']] = data[str(df.loc[i, 'node_id'])]
        


In [7]:
! pip install pickle5
import pickle5 as pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
# open and load data from pickle file
ner_embeddings_file = open('ner_embeddings.pkl', 'rb')

# keys are strings for node_id, value is embedding
ner_embeddings = pickle.load(ner_embeddings_file)

ner_embeddings_file.close()

In [9]:
subset= ner_embeddings.sample(n=1000)

In [10]:
subset

,NER,embeddings,num_ingredients,start_ingredient,embedding_matrix_index_list,start_ingredient_index
813442,"[egg, oil, sugar, vanilla, zucchini, pineapple...","[[0.25692862, 0.08700464, 0.23288268, 0.169938...",12,egg,"[2052, 4155, 5769, 6206, 6652, 4519, 2242, 276...",2052
2039943,"[flour, yeast, sugar, salt, oil, water]","[[-0.31055188, 0.15737705, -0.012394806, 0.129...",6,yeast,"[2242, 6600, 5769, 5228, 4155, 6355]",6600
1103950,"[almond, bacon, cheddar_cheese, mayonnaise, on...","[[0.18312205, -0.06159898, -0.2872927, 0.24822...",6,cheddar_cheese,"[79, 252, 1015, 3845, 4175, 5228]",1015
2188446,"[margarine, applesauce, flour, oat, sugar, bak...","[[-0.10213083, -0.28854743, -0.16905284, 0.134...",11,cinnamon,"[3806, 164, 2242, 4146, 5769, 277, 1285, 4140,...",1285
2163321,"[chicken_broth, baby_carrot, green_bean, cabba...","[[0.11032726, -0.22686784, -0.120354585, -0.00...",6,cabbage,"[1080, 221, 2792, 767, 5271, 2611]",767
...,...,...,...,...,...,...
1433791,"[sugar, sake, mirin, soy_sauce, fresh_ginger, ...","[[0.24356885, 0.20317887, -0.023231847, 0.0660...",7,fresh_ginger,"[5769, 5207, 3979, 5601, 2335, 6355, 1529]",2335
1439435,"[egg, sugar, cornstarch, milk, vanilla]","[[0.25692862, 0.08700464, 0.23288268, 0.169938...",5,cornstarch,"[2052, 5769, 1529, 3926, 6206]",1529
1423716,"[butter, cream_cheese, vanilla, kosher_salt, p...","[[-0.4366374, 0.026041824, 0.15680166, 0.11945...",8,milk,"[727, 1582, 6206, 3380, 4719, 3926, 913, 3926]",3926
1819926,"[cocoa, honey, margarine, splenda_sugar_substi...","[[0.0020710647, 0.017117582, -0.29813024, 0.25...",6,splenda_sugar_substitute,"[1338, 3084, 3806, 5665, 3822, 3929]",5665


In [11]:
# get processed data 
import pandas as pd
import re

# Next, we tokenize the data by creating a tokenizer object and fitting it to the cleaned data. 
# Tokenizer vectorizes the text corpus, turning it into a dictionary of words and their indexes.
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
corpus = subset['NER']

# For each review, we prepare n-grams to use as input sequences for training.
tokenizer.fit_on_texts(corpus)
input_sequences = []

for ingred_list in corpus:
	token_list = tokenizer.texts_to_sequences([ingred_list])[0]
	# print(token_list)
  # print(ingred_list)
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)


In [12]:
ingred_list

['margarine', 'flour', 'cream']

In [13]:
token_list

[15, 4, 76]

In [14]:
# Finally, we pad sequenced data and define predictors and labels. 
# We use predictors to guess what is the next word in a sequence and labels 
# to correct the model’s predictions.
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow.keras.utils as ku

# pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
total_words = len(tokenizer.word_index) + 1
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = ku.to_categorical(label, num_classes=total_words)

In [15]:
# Train the model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
import tensorflow as tf

In [16]:
model = Sequential()
model.add(Embedding(total_words, 240, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [17]:
# class myCallback(tf.keras.callbacks.Callback):
# 	def on_epoch_end(self, epoch, logs={}):
# 		if(logs.get('accuracy')>0.93):
# 			print("\nReached 93% accuracy so cancelling training!")
# 			self.model.stop_training = True

# callbacks = myCallback()

# the tutorial says that accuracy definitely gets better over time, they ran for 300 epochs but it just takes forever
# and we need to balance between the size of the subset vs the # of epochs so this is not a lot of train time relatively
# we could run it for longer once we get models we actually like
history = model.fit(predictors, label, epochs=100, verbose=1)


Epoch 1/100
194/194 [==============================] - 51s 218ms/step - loss: 5.8964 - accuracy: 0.0641
Epoch 2/100
194/194 [==============================] - 32s 165ms/step - loss: 5.2203 - accuracy: 0.0647
Epoch 3/100
194/194 [==============================] - 32s 164ms/step - loss: 5.0656 - accuracy: 0.0657
Epoch 4/100
194/194 [==============================] - 33s 172ms/step - loss: 4.9683 - accuracy: 0.0704
Epoch 5/100
194/194 [==============================] - 32s 165ms/step - loss: 4.9048 - accuracy: 0.0837
Epoch 6/100
194/194 [==============================] - 32s 165ms/step - loss: 4.8590 - accuracy: 0.0888
Epoch 7/100
194/194 [==============================] - 32s 166ms/step - loss: 4.8231 - accuracy: 0.0966
Epoch 8/100
194/194 [==============================] - 34s 176ms/step - loss: 4.7884 - accuracy: 0.0992
Epoch 9/100
194/194 [==============================] - 32s 166ms/step - loss: 4.7490 - accuracy: 0.1018
Epoch 10/100
194/194 [==============================] - 34s 174m

In [18]:
# class myCallback(tf.keras.callbacks.Callback):
# 	def on_epoch_end(self, epoch, logs={}):
# 		if(logs.get('accuracy')>0.93):
# 			print("\nReached 93% accuracy so cancelling training!")
# 			self.model.stop_training = True

# callbacks = myCallback()

# history = model.fit(predictors, label, epochs=300, verbose=1, callbacks=[callbacks])


In [41]:
seed_text = "water"
seed_text_two = "olive oil"
seed_text_three = "cinnamon"
seed_text_four = "garlic"
seed_text_three = "tequila"

In [21]:
# X_train, X_val, y_train, y_val= train_test_split(subset, subset['embeddings'], test_size= 0.2)

In [22]:
# EMBEDDING_DIMENSIONS = 300

# def construct_embedding_matrix(embeddings):
#     num_ing = len(embeddings)

#     # initialize a matrix of zeros
#     embedding_matrix = np.zeros((num_ing, EMBEDDING_DIMENSIONS)) # each embedding has 300 dimensions

#     next_row = 0

#     for i in embeddings:
#         v = embeddings.get(i)
#         embedding_matrix[next_row] = v
#         next_row+=1
    
#     return embedding_matrix

In [23]:
# embedding_matrix = construct_embedding_matrix(ing_embeddings)

In [24]:
# embedding_matrix

array([[-0.10600116,  0.04714949,  0.10841199, ..., -0.03144248,
        -0.06629407, -0.1286629 ],
       [-0.01582931,  0.09736368, -0.00062261, ..., -0.09226537,
        -0.12149926, -0.12204846],
       [-0.10132008,  0.03372396,  0.06472784, ..., -0.22692445,
        -0.04366636, -0.20344618],
       ...,
       [-0.19128327,  0.17544127, -0.09963894, ..., -0.20900002,
        -0.17799097, -0.1547064 ],
       [ 0.02008764,  0.04900858, -0.26409724, ..., -0.19495088,
        -0.16633987, -0.21576235],
       [ 0.20899913, -0.15171458, -0.25460058, ..., -0.18800448,
        -0.08664556, -0.07758268]])

In [25]:
# from keras import backend as K

# def recall_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     recall = true_positives / (possible_positives + K.epsilon())
#     return recall

# def precision_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     return precision

# def f1_m(y_true, y_pred):
#     precision = precision_m(y_true, y_pred)
#     recall = recall_m(y_true, y_pred)
#     return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [26]:
# MAX_SEQUENCE_LENGTH= 100

In [27]:
# from keras.models import Sequential
# from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D, Dropout
# from keras.initializers import Constant
# from keras.optimizers import Adam

In [28]:
# #print(type(X_train['start_ingredient']))
# import numpy as np

# x_train_array= X_train
# x_train_array= np.asarray(x_train_array)

# #print(type(v), v)

# x_val_array= X_val
# x_val_array= np.asarray(x_val_array)

# y_train= np.asarray(y_train)
# y_val= np.asarray(y_val)
# print(type(y_train))

<class 'numpy.ndarray'>


In [29]:
# [print(i.shape, i.dtype) for i in model.inputs]
# [print(o.shape, o.dtype) for o in model.outputs]
# [print(l.name, l.input_shape, l.dtype) for l in model.layers]

(None, 21) <dtype: 'float32'>
(None, 832) <dtype: 'float32'>
embedding (None, 21) float32
bidirectional (None, 21, 240) float32
dropout (None, 21, 300) float32
lstm_1 (None, 21, 300) float32
dense (None, 100) float32
dense_1 (None, 416) float32


[None, None, None, None, None, None]

In [30]:
# model=Sequential()

# embedding=Embedding(len(ing_embeddings), # number of unique tokens
#                     EMBEDDING_DIMENSIONS, #number of features
#                     embeddings_initializer=Constant(embedding_matrix), # initialize 
#                     input_length=MAX_SEQUENCE_LENGTH, 
#                     trainable=False)

# model.add(embedding)
# model.add(Dropout(0.2))
# model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.5))
# model.add(Dense(1, activation='sigmoid'))
# # compile the model
# optimzer = Adam(clipvalue=0.5) # clip value to avoid the gradient exploding

# model.compile(optimizer=optimzer, 
#               loss='binary_crossentropy', 
#               metrics=['acc',f1_m,precision_m, recall_m])

# # fit the model
# history = model.fit(x_train_array, y_train, 
#                     batch_size=32, 
#                     epochs=20, 
#                     validation_data=(x_val_array,y_val), 
#                     verbose=2)

ValueError: ignored